In [ ]:
import pandas as pd
import os

In [ ]:
# Distanza di Manhattan
# https://it.wikipedia.org/wiki/Geometria_del_taxi
def manhattan_distance(x, y):
    return sum(abs(a-b) for a,b in zip(x,y))
# Distanza Euclidea
# https://it.wikipedia.org/wiki/Distanza_euclidea
def euclidean_distance(x, y):
    return sum((a-b)**2 for a, b in zip(x, y))**0.5

# get specific columns from the dataset
columns_needed =  [
    'datetime',
    'Neck.px', 'Neck.py', 'Neck.pz',
    'ShoulderLeft.px', 'ShoulderLeft.py', 'ShoulderLeft.pz',
    'ShoulderRight.px', 'ShoulderRight.py', 'ShoulderRight.pz',
    'ElbowLeft.px', 'ElbowLeft.py', 'ElbowLeft.pz',
    'ElbowRight.px', 'ElbowRight.py', 'ElbowRight.pz',
    'WristLeft.px', 'WristLeft.py', 'WristLeft.pz',
    'WristRight.px', 'WristRight.py', 'WristRight.pz',
    'HipLeft.px', 'HipLeft.py', 'HipLeft.pz',
    'HipRight.px', 'HipRight.py', 'HipRight.pz',
    'KneeLeft.px', 'KneeLeft.py', 'KneeLeft.pz',
    'KneeRight.px', 'KneeRight.py', 'KneeRight.pz',
    'AnkleLeft.px', 'AnkleLeft.py', 'AnkleLeft.pz',
    'AnkleRight.px', 'AnkleRight.py', 'AnkleRight.pz'
]

In [ ]:
# get all the csv files from the dataset that are related to a specific movement
# each movement may have multiple repetitions
def get_movement_files(m):
    directory = '/home/Lucian/Github/Thesis/FoF-Analysis-Viz/dataset'
    files = {}
    for patients in os.listdir(directory):
        f = os.path.join(directory, patients)
        if patients.startswith('P'):
            if patients not in files:
                files[patients] = []
        if os.path.isdir(f):
            for movement in os.listdir(f):
                g = os.path.join(f, movement)
                if os.path.isdir(g) and movement == m:
                    for repetition in os.listdir(g):
                        h = os.path.join(g, repetition)
                        if os.path.isdir(h):
                            for file in os.listdir(h):
                                if file.endswith('.csv'):
                                    result = os.path.join(h, file)
                                    files[patients].append(result)     
    # sort the patients by the endidng number of their name
    files = dict(sorted(files.items(), key=lambda item: item[0][1:]))           
    return files
# list the files 
def list_files(files):
    for patient, movements in files.items():
        print(patient)
        for movement in movements:
            print('\t', movement)
# get the dataframes from each file and keep only the columns needed 
def get_dataframe_from_file(files, colums_needed):
   dataframes = {}
   for patient, movements in files.items():
       if patient not in dataframes:
           dataframes[patient] = []
       for movement in movements:
           df = pd.read_csv(movement, sep=';')
           df = df[columns_needed]
           dataframes[patient].append(df)
   return dataframes         
           
def get_tuple_from_row(df, x, y, z):
    return df[[x, y, z]].apply(tuple, axis=1)

def get_values_from_df(dataframes):
    values = {}
    for patient, movements in dataframes.items():
        if patient not in values:
            values[patient] = []
        for movement in movements:
            result = get_tuple_from_row(movement, 'Neck.px', 'Neck.py', 'Neck.pz')
            if result is not None:
                values[patient].append(result)
    return values

files = get_movement_files('M000')  
dataframes = get_dataframe_from_file(files, columns_needed)
values  = get_values_from_df(dataframes)

In [54]:
def calculate_manhattan_distance(values):
    distances = {}
    for patient, movements in values.items():
        if patient not in distances:
            distances[patient] = []
        for i in range(1, 50):
            point1 = movements[i-1]
            point2 = movements[i]
            distance = manhattan_distance(point1, point2)
            distances[patient].append(distance)

calculate_manhattan_distance(values)

0     (-1.787606, -0.111566, 3.379106)
1      (-1.762118, -0.15765, 3.413362)
2       (-1.757094, -0.2012, 3.427075)
3     (-1.795848, -0.078952, 3.385777)
4     (-1.786978, -0.142356, 3.380571)
                    ...               
65    (-1.789727, -0.113758, 3.393225)
66    (-1.787149, -0.141098, 3.394675)
67    (-1.800803, -0.099011, 3.393462)
68    (-1.786219, -0.138434, 3.378013)
69     (-1.801359, -0.12004, 3.396053)
Length: 70, dtype: object
0      (-1.789848, -0.121837, 3.400296)
1       (-1.791599, -0.124737, 3.37992)
2      (-1.788346, -0.140067, 3.390156)
3      (-1.787849, -0.200341, 3.407406)
4       (-1.79511, -0.115847, 3.385478)
                     ...               
185    (-1.797602, -0.087593, 3.391656)
186    (-1.797027, -0.090633, 3.396003)
187     (-1.788271, -0.123996, 3.39259)
188     (-1.789248, -0.124175, 3.39436)
189     (-1.777804, -0.14749, 3.436823)
Length: 190, dtype: object
0      (-0.770677, 0.248295, 2.754511)
1       (-0.768506, 0.253256, 2.76661)
